## ライブラリのインポート

In [1]:
!pip install pymysql
!pip install janome

In [2]:
import pandas as pd
import numpy as np
import ast
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import math

import pymysql
import re
import janome
import warnings
warnings.filterwarnings('ignore')

## データの読み込み
---

In [3]:
 sql = """
SELECT 
    mas_j.company_id
    , mas_j.position
    , mas_j.job_board_info
    , lang_s.language_skillset
    , fram_s.framework_skillset
    , work_c.working_conditions
FROM
    {table_pre}_master_job_board AS mas_j
    LEFT JOIN 
        {table_pre}_language_skillset AS lang_s
        ON mas_j.company_id = lang_s.company_id
    LEFT JOIN {table_pre}_framework_skillset AS fram_s
        ON mas_j.company_id = fram_s.company_id
    LEFT JOIN {table_pre}_working_conditions AS work_c
        ON mas_j.company_id = work_c.company_id
"""

In [4]:
#sqlに接続はセキュリティ上省略

In [5]:
# sql文内のtable_preにtrainを代入します。
# DBから取り出したデータ中の使用したいカラム名を配列col_namesに格納します。
train_sql = sql.format(table_pre="train")
col_names = ["company_id", "position", "job_board_info",
              "language_skillset", "framework_skillset","working_conditions"]

# DBから得たデータをtrain_dataに入れます。
train_data = fetch_data(train_sql, col_names)
train_data.head()

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions
0,Cd346dca,サーバサイドエンジニア（開発チーム リーダー候補）,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/ Python3/ JavaScript,CakePHP/ jQuery,服装自由/ イヤホンOK/ ノートPC＋モニタ別途支給/ 残業３０H以内/ フリードリンク/...
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/ C/ VB/ JavaScript/ HTML+CSS,,残業３０H以内/ 日本語がネイティブレベルでなくても可
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/ PHP/ Ruby/ HTML5+CSS3/ HTML+CSS,CodeIgniter/ Ruby on Rails,服装自由/ イヤホンOK/ ノートPC＋モニタ別途支給/ 裁量労働/ フリードリンク/ 若手...
3,C10010c0,バックエンドエンジニア（大阪）,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/ JavaScript/ HTML5+CSS3/ HTML+CSS/ SQL,FuelPHP/ jQuery/ Vue.js,服装自由/ 社長が現役or元エンジニア/ フリーソフト利用可/ ノートPC＋モニタ別途支給/...
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/ Ruby/ JavaScript/ HTML5+CSS3/ SQL,CodeIgniter/ Smarty/ Ruby on Rails/ jQuery,服装自由/ イヤホンOK/ フリーソフト利用可/ ノートPC＋モニタ別途支給/ 1920x1...


In [6]:
import re
def get_manegement(df,col_name):

  train_data["manegement"] =0
  rows = df[col_name]
  # replaceメソッドを使って不要な文字を削除します。"\u3000"は全角空白を表します。


  for i, row in enumerate(rows):
    # if re.search("マネージャー" | "リーダー" | "PM" | "PL" | "CTO" ,row):
    if re.search("(マネージャー)|(リーダー)|(PM)|(PL)|(CTO)" ,row):
      train_data["manegement"][i]=1
    else:
      
      train_data["manegement"][i]=0
 
    
    
      
    

get_manegement(train_data,"position")

    

### ヒント：列名job_board_infoから年salary_minを抽出する方法

In [7]:
def parse_job_board_info(df, data_type):
    employee_num = []
    location = []
    salary_min = []

    # データタイプがtrainの時の処理
    if data_type == "train":
        for record in df["job_board_info"]:
            record= record.replace("nan","0")
            recordict = ast.literal_eval(record)
            employee_num.append(recordict["employee_num"])
            location.append(recordict["location"])
            salary_min.append(recordict["salary_min"])

        return employee_num, location, salary_min

    # データタイプがtestの時の処理
    if data_type == "test":
        for record in df["job_board_info"]:
            record = record.replace("nan", "0")
            # 辞書型をした文字列をastライブラリのliteral_evalメソッドで辞書型のデータに変換します。
            recordict = ast.literal_eval(record)
            # 変換した辞書からemployee_num、locationを抽出し、戻り値に渡します。
            # テスト用のデータなので"salary_min"がないことに注意してください。
            employee_num.append(recordict["employee_num"])
            location.append(recordict["location"])
        return employee_num, location

In [8]:
employee_num, location, salary_min = parse_job_board_info(train_data, "train")

# 取得したデータをtrain_dataに追加します。
train_data["employee_num"] = employee_num
train_data["location"] = location
train_data["salary_min"] = salary_min

train_data.head()

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min
0,Cd346dca,サーバサイドエンジニア（開発チーム リーダー候補）,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/ Python3/ JavaScript,CakePHP/ jQuery,服装自由/ イヤホンOK/ ノートPC＋モニタ別途支給/ 残業３０H以内/ フリードリンク/...,1,26.0,東京都千代田区,510.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/ C/ VB/ JavaScript/ HTML+CSS,,残業３０H以内/ 日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/ PHP/ Ruby/ HTML5+CSS3/ HTML+CSS,CodeIgniter/ Ruby on Rails,服装自由/ イヤホンOK/ ノートPC＋モニタ別途支給/ 裁量労働/ フリードリンク/ 若手...,0,76.0,東京都目黒区,400.0
3,C10010c0,バックエンドエンジニア（大阪）,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/ JavaScript/ HTML5+CSS3/ HTML+CSS/ SQL,FuelPHP/ jQuery/ Vue.js,服装自由/ 社長が現役or元エンジニア/ フリーソフト利用可/ ノートPC＋モニタ別途支給/...,0,53.0,大阪府大阪市,350.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/ Ruby/ JavaScript/ HTML5+CSS3/ SQL,CodeIgniter/ Smarty/ Ruby on Rails/ jQuery,服装自由/ イヤホンOK/ フリーソフト利用可/ ノートPC＋モニタ別途支給/ 1920x1...,1,30.0,東京都新宿区,600.0


### ヒント：positionをダミー変数化する方法

In [9]:
del_dic = {
    "'": "",
    " ": "",
    "[": "",
    "]": "",
    ",": "/"
}


# translateメソッドを使って、文字を変換する関数を定義
def del_str(x):
    dic_for_unify = str.maketrans(del_dic)
    x = x.translate(dic_for_unify)
    return x


# del_str関数を適用したいカラムを配列で指定し、実行します。
modified_columns = ["framework_skillset",
                    "language_skillset", "working_conditions"]

for col in modified_columns:
    train_data[col] = train_data[col].apply(del_str)

In [10]:
train_data

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min
0,Cd346dca,サーバサイドエンジニア（開発チーム リーダー候補）,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0
3,C10010c0,バックエンドエンジニア（大阪）,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0
...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア（PL）,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0
1200,C1893d63,サーバーサイドエンジニア（PL）,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0


In [11]:
def get_element_list(df, col_name):
    # joinメソッドによりカラムの要素を"/"で繋げます。
    all_ele = "/".join(df[col_name])
    # splitメソッドにより文字列を"/"区切り、リスト化します。
    all_ele_list = all_ele.split("/")
    # .strip(): 先頭・末尾の空白を削除する
    all_ele_list = [ele.strip() for ele in all_ele_list]
    return all_ele_list


frame_skill = get_element_list(train_data, "framework_skillset")

# setメソッドでリスト内の要素の重複をなくします。
unique_fram_list = list(set(frame_skill))
unique_fram_list[:10]

['',
 'Caffe',
 'DirectX',
 'iOSSDK',
 'Catalyst',
 'Sinatra',
 'cocos2d-x',
 'Struts',
 'ZendFramework',
 'cocos2d']

In [12]:
train_data

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min
0,Cd346dca,サーバサイドエンジニア（開発チーム リーダー候補）,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0
3,C10010c0,バックエンドエンジニア（大阪）,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0
...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア（PL）,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0
1200,C1893d63,サーバーサイドエンジニア（PL）,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0


In [13]:
def get_dummy_mat(df, unique_list, col_name):
    # DataFrameの行数×重複を消した配列の長さのゼロ行列を作成します。
    dummy_mat = np.zeros((df.shape[0],  len(unique_list)))

    # 各レコード内のあるカラム(例えば、100番目)に特定のスキル(例えば、unique_list[10]に該当するスキル)があった場合、ゼロ行列の[100,10]に1を代入します。
    for i, row in enumerate(df[col_name]):
        tmp = row.split("/")
        for one in tmp:
            # stripメソッドで空白や改行を取り除きます。
            one = one.strip()
            if one in unique_list:
                idx = unique_list.index(one)
                dummy_mat[i, idx] = 1
    return dummy_mat


# "framework_skillset"についてのダミー変数を作成します。
frame_dummy = get_dummy_mat(train_data, unique_fram_list, "framework_skillset")
frame_df = pd.DataFrame(frame_dummy, columns=unique_fram_list)
frame_df.head()

,,Caffe,DirectX,iOSSDK,Catalyst,Sinatra,cocos2d-x,Struts,ZendFramework,cocos2d,...,UnrealEngine,.NETFramework,FuelPHP,Spring,Seasar2,JSF,React,TensorFlow,Unity,iBATIS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
frame_df.drop(columns="", inplace=True)

In [15]:
def get_frame_dummy(data, unique_fram_list):
    frame_dummy = get_dummy_mat(data, unique_fram_list, "framework_skillset")
    frame_df = pd.DataFrame(frame_dummy, columns=unique_fram_list)
    frame_df.drop(columns="", inplace=True)
    return frame_df

In [16]:
# "environment_skillset"カラムの要素を被りなく抽出します。
env_skill = get_element_list(train_data, "language_skillset")
env_skill[:10]

['PHP',
 'Python3',
 'JavaScript',
 'Java',
 'C',
 'VB',
 'JavaScript',
 'HTML+CSS',
 'Go言語',
 'PHP']

In [17]:
def get_most_common(ele_list, topN):
    # collectionsライブラリのCounterメソッドで各要素の出現回数をカウントできます。
    common_ele = collections.Counter(ele_list)
    # most_commonを用いることで出現回数順に並べたリストを出力することができます。
    topN_common_ele = common_ele.most_common()[:topN]
    topN_common_ele_list = [ele[0] for ele in topN_common_ele]
    return topN_common_ele_list


top_env = get_most_common(env_skill, 40)
top_env[:10]

['JavaScript',
 'Java',
 'PHP',
 'SQL',
 'HTML5+CSS3',
 'Ruby',
 'C＃',
 'Python3',
 'HTML+CSS',
 'C++']

In [18]:
env_dummy = get_dummy_mat(train_data, top_env, "language_skillset")
env_df = pd.DataFrame(env_dummy, columns=top_env)
env_df.head()

,JavaScript,Java,PHP,SQL,HTML5+CSS3,Ruby,C＃,Python3,HTML+CSS,C++,...,ActionScript,CUDAC,Erlang,Groovy,Lua,ECMAScript,Rust,Clojure,Elixir,Nim
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
env_df.drop(columns="", inplace=True)

In [20]:
# testデータ用にダミー変数の作成と空白の列を削除する関数を作成します。
def get_env_dummy(data, top_env):
    env_dummy = get_dummy_mat(data, top_env, "language_skillset")
    env_df = pd.DataFrame(env_dummy, columns=top_env)
    env_df.drop(columns="", inplace=True)
    return env_df

In [21]:
re.sub("\（.*?\）|\(.*?\)", "", "SRE（Site Reliability Engineering）")

'SRE'

In [22]:
# 丸括弧で囲まれた部分を削除する関数
def del_brackets(row):
    # 正規表現
    row = re.sub("\（.*?\）|\(.*?\)", "", row)
    return row


# positionカラムに対してdel_brackets関数を適用します。
train_data["position"] = train_data["position"].apply(del_brackets)

In [23]:
train_data


,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min
0,Cd346dca,サーバサイドエンジニア,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0
3,C10010c0,バックエンドエンジニア,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0
...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0
1200,C1893d63,サーバーサイドエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0


In [24]:
def get_unique_position(df, col_name):
    rows = df[col_name]
    # replaceメソッドを使って不要な文字を削除します。"\u3000"は全角空白を表します。
    ele_list = [ele.replace("\u3000", "/") for ele in rows]
    all_ele_list = "/".join(ele_list)
    all_ele_list = all_ele_list.split("/")
    unique_ele_list = list(set(all_ele_list))
    return unique_ele_list


position = get_unique_position(train_data, "position")
position[:40]

['',
 'リーダー',
 'プロダクトマネージャー',
 'RPAエンジニア',
 'マネージャー',
 'リードエンジニア候補',
 'Webシステムエンジニア',
 '初級SE',
 'iOSアプリエンジニア',
 'Blockchain アーキテクト',
 'IoT車載系エンジニア',
 'OSS基盤エンジニア',
 'プログラマー、システムエンジニア、プロジェクトマネージャ',
 'テクニカルディレクター',
 '【長野県勤務】システムエンジニア・プログラマー',
 'スマートフォンアプリ開発リードエンジニア',
 'ドメイン駆動設計を実践する Java エンジニア',
 'AIエンジニア',
 'パッケージソフトの開発及び保守',
 'システム構築エンジニア',
 'Androidエンジニアリングマネージャー',
 'Railsアプリケーションエンジニア',
 'フルスタック型リードエンジニア',
 'インテグレーションエンジニア',
 'IoTプラットフォーム開発エンジニア',
 'フロントエンドエンジニアリーダー',
 'PG、SE、PL',
 ' VPoE',
 'Webエンジニア ',
 '開発',
 'AWSエンジニア',
 '組込み系システムエンジニア・プログラマ',
 'ブロックチェーンエンジニア',
 'デジタルマーケティングエンジニア',
 '【自動運転・ADAS】組み込みｘ画像処理アルゴリズム',
 'データ分析エンジニア、データアナリスト、データサイエンティスト',
 'PL／PM',
 '開発マネージャー候補',
 'リードアプリエンジニア',
 'アプリケーション開発サブリーダー・リーダー']

In [25]:
posi_dummy = get_dummy_mat(train_data, position, "position")
posi_df = pd.DataFrame(posi_dummy, columns=position, dtype=int)
posi_df.head()

,,リーダー,プロダクトマネージャー,RPAエンジニア,マネージャー,リードエンジニア候補,Webシステムエンジニア,初級SE,iOSアプリエンジニア,Blockchain アーキテクト,...,SDKエンジニア,WEB・スマホアプリ等の開発エンジニア,Androidアプリエンジニア,コアエンジニア,railsリードエンジニア,Web・スマホアプリ開発エンジニア,IoTエンジニア,自社開発 AIプロダクト開発リーダー,UI／UXエンジニア,サービスリード、テックリード
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
posi_df.drop(columns="", inplace=True)

In [27]:
# testデータ用にダミー変数の作成と空白の列を削除する関数を作成します。
def get_posi_dummy(data, position):
    posi_dummy = get_dummy_mat(data, position, "position")
    posi_df = pd.DataFrame(posi_dummy, columns=position, dtype=int)
    posi_df.drop(columns="", inplace=True)
    return posi_df

In [28]:
from janome.tokenizer import Tokenizer

In [29]:
addresses = list(train_data.location)
addresses

['東京都千代田区',
 '東京都立川市',
 '東京都目黒区',
 '大阪府大阪市',
 '東京都新宿区',
 '東京都港区',
 '東京都渋谷区',
 '愛知県名古屋市',
 '東京都千代田区',
 '北海道札幌市',
 '岡山県北区',
 '東京都渋谷区',
 '東京都文京区',
 '東京都港区',
 '大阪府中央区',
 '大阪府中央区',
 '東京都千代田区',
 '東京都千代田区',
 '東京都港区',
 '大阪府大阪市',
 '東京都目黒区',
 '東京都新宿区',
 '東京都新宿区',
 '愛知県名古屋市',
 '東京都港区',
 '東京都千代田区',
 '東京都千代田区',
 '東京都千代田区',
 '東京都品川区',
 '東京都千代田区',
 '大阪府大阪市',
 '東京都千代田区',
 '東京都千代田区',
 '東京都目黒区',
 '東京都千代田区',
 '東京都千代田区',
 '愛知県名古屋市',
 '沖縄県豊見城市',
 '東京都新宿区',
 '東京都港区',
 '東京都渋谷区',
 '東京都新宿区',
 '東京都渋谷区',
 '東京都千代田区',
 '東京都渋谷区',
 '東京都中央区',
 '東京都港区',
 '東京都新宿区',
 '東京都渋谷区',
 '東京都中央区',
 '東京都渋谷区',
 '東京都中央区',
 '東京都渋谷区',
 '愛知県名古屋市',
 '東京都新宿区',
 '東京都中央区',
 '東京都渋谷区',
 '愛知県名古屋市',
 '東京都中央区',
 '東京都港区',
 '東京都千代田区',
 '東京都千代田区',
 '東京都港区',
 '東京都新宿区',
 '東京都文京区',
 '東京都新宿区',
 '東京都世田谷区',
 '東京都新宿区',
 '東京都新宿区',
 '東京都新宿区',
 '東京都新宿区',
 '東京都港区',
 '北海道札幌市',
 '北海道札幌市',
 '大阪府大阪市',
 '兵庫県神戸市',
 '東京都新宿区',
 '東京都新宿区',
 '東京都新宿区',
 '東京都渋谷区',
 '東京都港区',
 '東京都目黒区',
 '東京都恵比寿南',
 '東京都渋谷区',
 '福岡県福岡市',
 '東京都千代田区',
 '東京都目黒区',
 '滋賀県大津市',
 '東京都千代田区',
 '東京都品川

In [30]:
def get_pref(add_list):
    pref = []
    t = Tokenizer(wakati=False)
    for address in add_list:
        address_row = [i.surface for i in t.tokenize(address)]
        if len(address_row) > 0:
            
            pref.append(address_row[0])
    return pref

pref_list = get_pref(addresses)
train_data["pref"] = pref_list

In [31]:
# ダミー変数用に重複なし都道府県情報を作成します。
unique_region = list(set(pref_list))

# "pref"についてのダミー変数を作成します。
pref_dummy = get_dummy_mat(train_data, unique_region, "pref")
pref_df = pd.DataFrame(pref_dummy, columns=unique_region)
pref_df.head()

,山梨,新潟,大分,福岡,和歌山,埼玉,茨城,広島,長野,東京,...,静岡,香川,神奈川,宮崎,滋賀,京都,宮城,北海道,千葉,愛知
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
train_data

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min,pref
0,Cd346dca,サーバサイドエンジニア,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0,東京
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0,東京
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0,東京
3,C10010c0,バックエンドエンジニア,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0,大阪
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0,東京
...,...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,東京
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,東京
1200,C1893d63,サーバーサイドエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,東京
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,東京


In [33]:
def get_pref_dummy(data, unique_region):
    addresses = list(data.location)
    pref_list = get_pref(addresses)
    data["pref"] = pref_list
    pref_dummy = get_dummy_mat(data, unique_region, "pref")
    pref_df = pd.DataFrame(pref_dummy, columns=unique_region)
    return pref_df, addresses

In [34]:
# "working_conditions"カラムの要素を被りなく抽出します。
work_con = get_element_list(train_data, "working_conditions")
unique_work_con = list(set(work_con))

In [35]:
unique_work_con 

['',
 'スキル研修が充実',
 '若手歓迎',
 '残業３０H以内',
 'OSSプロジェクトのコミッタが在籍',
 '女性エンジニアが在籍',
 'ノートPC＋モニタ別途支給',
 '経験が浅い方歓迎',
 '一部在宅勤務可',
 'フリードリンク',
 '第二新卒歓迎',
 '社長が現役or元エンジニア',
 '女性リーダー活躍中',
 '1920x1200以上のモニター環境を提供',
 '産休育休取得実績あり',
 '時短勤務可',
 'オンライン面談可',
 '服装自由',
 'フリーソフト利用可',
 '原則定時退社',
 '既卒可',
 'イヤホンOK',
 '裁量労働',
 '副業OK',
 '日本語がネイティブレベルでなくても可']

In [36]:
# "working_condition"についてのダミー変数を作成します。
word_dummy = get_dummy_mat(train_data, unique_work_con, "working_conditions")
work_df = pd.DataFrame(word_dummy, columns=unique_work_con)
work_df.head()

,,スキル研修が充実,若手歓迎,残業３０H以内,OSSプロジェクトのコミッタが在籍,女性エンジニアが在籍,ノートPC＋モニタ別途支給,経験が浅い方歓迎,一部在宅勤務可,フリードリンク,...,時短勤務可,オンライン面談可,服装自由,フリーソフト利用可,原則定時退社,既卒可,イヤホンOK,裁量労働,副業OK,日本語がネイティブレベルでなくても可
0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [37]:
work_df.drop(columns="", inplace=True)

In [38]:
# testデータ用にダミー変数の作成と空白の列を削除する関数を作成します。
def get_work_dummy(data, unique_work_con):
    word_dummy = get_dummy_mat(data, unique_work_con, "working_conditions")
    work_df = pd.DataFrame(word_dummy, columns=unique_work_con)
    work_df.drop(columns="", inplace=True)
    return work_df

In [39]:
# DataFrameを結合する関数を作成
def merge_dummies(data, frame_df, env_df, pref_df, posi_df, work_df):
    merged_data = pd.merge(data, frame_df, left_index=True, right_index=True)
    merged_data = pd.merge(merged_data, env_df, left_index=True, right_index=True)
    merged_data = pd.merge(merged_data, pref_df, left_index=True, right_index=True)
    merged_data = pd.merge(merged_data, posi_df, left_index=True, right_index=True)
    merged_data = pd.merge(merged_data, work_df, left_index=True, right_index=True)
    return merged_data

In [40]:
train = merge_dummies(train_data, frame_df, env_df, pref_df, posi_df, work_df)
train.head()

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min,...,時短勤務可,オンライン面談可,服装自由,フリーソフト利用可,原則定時退社,既卒可,イヤホンOK,裁量労働,副業OK,日本語がネイティブレベルでなくても可
0,Cd346dca,サーバサイドエンジニア,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,C10010c0,バックエンドエンジニア,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [41]:
def get_manegement_test(df,col_name):

  test_data["manegement"] =0
  rows = df[col_name]
  # replaceメソッドを使って不要な文字を削除します。"\u3000"は全角空白を表します。


  for i, row in enumerate(rows):
    # if re.search("マネージャー" | "リーダー" | "PM" | "PL" | "CTO" ,row):
    if re.search("(マネージャー)|(リーダー)|(PM)|(PL)|(CTO)" ,row):
      test_data["manegement"][i]=1
    else:
      
      test_data["manegement"][i]=0

In [42]:
test_sql = sql.format(table_pre="test")
col_names = ["company_id", "position", "job_board_info",
             "language_skillset", "framework_skillset", "working_conditions"]

test_data = fetch_data(test_sql, col_names)

In [43]:

def check_blank(data):
    for col in data.columns:
        blank_data = (data[train_data[col] == ""])
        print(col, blank_data.shape[0])




In [44]:
check_blank(test_data)

company_id 0
position 0
job_board_info 0
language_skillset 7
framework_skillset 106
working_conditions 1


In [45]:
employee_num, location = parse_job_board_info(test_data, "test")
get_manegement_test(test_data,"position")
test_data["employee_num"] = employee_num
test_data["location"] = location

modified_columns = ["framework_skillset",
                    "language_skillset", "working_conditions"]
for col in modified_columns:
    test_data[col] = test_data[col].apply(del_str)

frame_df = get_frame_dummy(test_data, unique_fram_list)
env_df = get_env_dummy(test_data, top_env)
posi_df = get_posi_dummy(test_data, position)
pref_df, test_addresses = get_pref_dummy(test_data, unique_region)
work_df = get_work_dummy(test_data, unique_work_con)

test = merge_dummies(test_data, frame_df, env_df, pref_df, posi_df, work_df)
test.head()

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,pref,...,時短勤務可,オンライン面談可,服装自由,フリーソフト利用可,原則定時退社,既卒可,イヤホンOK,裁量労働,副業OK,日本語がネイティブレベルでなくても可
0,C1896105,ネットワーク・エンジニア（東京）,"{'employee_num': 24.0, 'location': '東京都中央区'}",Java/PHP/Perl/JavaScript/SQL,Smarty/jQuery,服装自由/残業３０H以内/オンライン面談可,0,24.0,東京都中央区,東京,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1897404,画像処理ソフトウェアエンジニア,"{'employee_num': 176.0, 'location': '東京都品川区'}",C++/Java/C,,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,176.0,東京都品川区,東京,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
2,C1898404,ソフトウェア高速化エンジニア,"{'employee_num': 176.0, 'location': '東京都品川区'}",C++/Java/C,,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,176.0,東京都品川区,東京,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
3,C1899404,低レイヤ技術エンジニア,"{'employee_num': 176.0, 'location': '東京都品川区'}",C++/C/Ruby/Perl,,イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/1...,0,176.0,東京都品川区,東京,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
4,C3795f64,クライアントエンジニア（Unreal Engine 4）,"{'employee_num': 300.0, 'location': '東京都港区'}",C++,UnrealEngine,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/1920x1200以上のモニター環境を提...,0,300.0,東京都港区,東京,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [46]:
train

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min,...,時短勤務可,オンライン面談可,服装自由,フリーソフト利用可,原則定時退社,既卒可,イヤホンOK,裁量労働,副業OK,日本語がネイティブレベルでなくても可
0,Cd346dca,サーバサイドエンジニア,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,C10010c0,バックエンドエンジニア,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1200,C1893d63,サーバーサイドエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [47]:
train

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min,...,時短勤務可,オンライン面談可,服装自由,フリーソフト利用可,原則定時退社,既卒可,イヤホンOK,裁量労働,副業OK,日本語がネイティブレベルでなくても可
0,Cd346dca,サーバサイドエンジニア,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,C10010c0,バックエンドエンジニア,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1200,C1893d63,サーバーサイドエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [48]:
def laguage_kouskill(df,col_name):
  train_data["skill"] =0
  rows = df[col_name]
  for i, row in enumerate(rows):
    all_ele = "/".join(df[col_name])
    # splitメソッドにより文字列を"/"区切り、リスト化します。
    all_ele_list = all_ele.split("/")
    # .strip(): 先頭・末尾の空白を削除する
    
    print(all_ele_list)
    
    
    
      
    

laguage_kouskill(train_data,"language_skillset")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




['PHP', 'Python3', 'JavaScript', 'Java', 'C', 'VB', 'JavaScript', 'HTML+CSS', 'Go言語', 'PHP', 'Ruby', 'HTML5+CSS3', 'HTML+CSS', 'PHP', 'JavaScript', 'HTML5+CSS3', 'HTML+CSS', 'SQL', 'PHP', 'Ruby', 'JavaScript', 'HTML5+CSS3', 'SQL', 'PHP', 'C++', 'C＃', 'JavaScript', 'HTML+CSS', 'Scala', 'Swift', 'JavaScript', 'HTML+CSS', 'Kotlin', 'Python3', 'C＃', 'Ruby', 'R言語', 'Python3', 'Python3', 'PHP', 'Python2', 'Python3', 'VB', 'JavaScript', 'HTML+CSS', 'Java', 'Python3', 'JavaScript', 'Kotlin', 'PHP', 'Python2', 'Python3', 'VB', 'JavaScript', 'HTML+CSS', 'Java', 'PHP', 'SQL', 'Bash', 'Ruby', 'JavaScript', 'Ruby', 'JavaScript', 'Java', 'C＃', 'PHP', 'JavaScript', 'HTML5+CSS3', 'Python3', 'Go言語', 'TypeScript', 'PHP', 'JavaScript', 'HTML5+CSS3', 'HTML+CSS', 'SQL', 'PHP', 'JavaScript', 'HTML5+CSS3', 'HTML+CSS', 'SQL', 'Java', 'PHP', 'SQL', 'JavaScript', 'HTML5+CSS3', 'HTML+CSS', 'TypeScript', 'PHP', 'Python3', 'Java', 'C', 'Ruby', 'R言語', 'Go言語', 'JavaScript', 'Ruby', 'Python3', 'PHP', 'Scala', 'Go言語'

In [49]:
train_data

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min,pref,skill
0,Cd346dca,サーバサイドエンジニア,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0,東京,0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0,東京,0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0,東京,0
3,C10010c0,バックエンドエンジニア,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0,大阪,0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0,東京,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,東京,0
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,東京,0
1200,C1893d63,サーバーサイドエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,東京,0
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,東京,0


In [50]:
# 機械学習を用いるためにライブラリを読み込みます。
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [51]:
# 目的変数となるカラム名を定義します。
target_col = "salary_min"

# 予測に必要ないカラム名を定義します。
exclude_col = [
    "salary_min",
    "company_id",
    "language_skillset",
    "framework_skillset",
    "location",
    "position",
    "working_conditions",
    "job_board_info",
    "pref"
]

# 分析に使うカラム名のみfeature_colsに追加していきます。
feature_cols = []
for col in train.columns:
    if col not in exclude_col:
        feature_cols.append(col)

In [52]:
# 作成したDataFrameを学習用やバリデーション用に分割します。
X_f = train[feature_cols]
y_f = train[target_col]

X_train_f, X_valid_f, y_train_f, y_valid_f = train_test_split(
    X_f, y_f, test_size=0.30, random_state=1234)

# testから予測に使うカラムのみ抽出します。
X_test_f = test[feature_cols]

In [53]:
from sklearn.ensemble import RandomForestRegressor

# モデルを初期化します。
simple_rf = RandomForestRegressor(random_state=1234)

# X_trainとy_trainを使ってモデルの学習します。
simple_rf.fit(X_train_f, y_train_f)

# 学習済みモデルとX_valid_fを使って予測します。
y_pred = simple_rf.predict(X_valid_f)

# 実際のデータであるy_valid_fと予測データとのRMSEを算出します。
RMSE = np.sqrt(mean_squared_error(y_valid_f, y_pred))

print(RMSE)
# 辞書型のデータを用意し、スコアを保存します。
RMSE_dic = {}

RMSE_dic["simple_rf"] = RMSE

77.83417716266158


In [54]:
y_pred = simple_rf.predict(X_test_f)
output = pd.DataFrame()
output["company_id"] = test["company_id"]
output["salary_min"] = y_pred
output.to_csv("sample_3.csv", index=False)

In [55]:
# lasso回帰のライブラリを読み込みます。
from sklearn.linear_model import Lasso

# モデルを初期化します。random_stateを固定します。
simple_lasso = Lasso(random_state=1234)

# X_trainとy_trainを使ってモデルの学習します。
simple_lasso.fit(X_train_f, y_train_f)

# 学習済みモデルとX_valid_fを使って予測します。
y_pred = simple_lasso.predict(X_valid_f)

# 実際のデータであるy_valid_fと予測データとのRMSEを算出します。
RMSE = np.sqrt(mean_squared_error(y_valid_f, y_pred))

print(RMSE)
RMSE_dic["simple_lasso"] = RMSE

78.49829937541972


In [56]:
import lightgbm as lgb 
lgb_train = lgb.Dataset(X_train_f, y_train_f)
lgb_eval = lgb.Dataset(X_valid_f, y_valid_f, reference=lgb_train) 

# LightGBM parameters
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression', # 目的 : 回帰  
        'metric': {'rmse'}, # 評価指標 : rsme(平均二乗誤差の平方根) 
}

# モデルの学習
model = lgb.train(params,
                  train_set=lgb_train, # トレーニングデータの指定
                  valid_sets=lgb_eval, # 検証データの指定
                  )

# テストデータの予測
y_pred = model.predict(X_valid_f)
RMSE = np.sqrt(mean_squared_error(y_valid_f, y_pred))

print(RMSE)

[1]	valid_0's rmse: 86.509
[2]	valid_0's rmse: 85.0932
[3]	valid_0's rmse: 83.9115
[4]	valid_0's rmse: 83.0186
[5]	valid_0's rmse: 82.1356
[6]	valid_0's rmse: 81.4054
[7]	valid_0's rmse: 80.9605
[8]	valid_0's rmse: 80.3479
[9]	valid_0's rmse: 79.838
[10]	valid_0's rmse: 79.4503
[11]	valid_0's rmse: 79.2592
[12]	valid_0's rmse: 78.9305
[13]	valid_0's rmse: 78.8324
[14]	valid_0's rmse: 78.5089
[15]	valid_0's rmse: 78.4082
[16]	valid_0's rmse: 78.2708
[17]	valid_0's rmse: 78.0891
[18]	valid_0's rmse: 77.9948
[19]	valid_0's rmse: 77.982
[20]	valid_0's rmse: 78.1065
[21]	valid_0's rmse: 78.0577
[22]	valid_0's rmse: 77.988
[23]	valid_0's rmse: 77.8313
[24]	valid_0's rmse: 77.6863
[25]	valid_0's rmse: 77.7923
[26]	valid_0's rmse: 77.8555
[27]	valid_0's rmse: 77.92
[28]	valid_0's rmse: 77.9784
[29]	valid_0's rmse: 78.0266
[30]	valid_0's rmse: 78.1608
[31]	valid_0's rmse: 78.0566
[32]	valid_0's rmse: 78.242
[33]	valid_0's rmse: 78.224
[34]	valid_0's rmse: 78.1279
[35]	valid_0's rmse: 78.0342
[3

In [57]:

f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
df_importance = pd.DataFrame({'feature':feature_cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

,feature,importance
1,employee_num,658
527,若手歓迎,124
528,残業３０H以内,114
54,PHP,96
53,Java_x,92
...,...,...
225,PHPエンジニア,0
224,ソフトウェア開発技術職,0
223,ヘルスケアIoT◎マネージャー候補,0
222,アプリ開発エンジニア,0


In [58]:
coef_df = pd.DataFrame(
    simple_lasso.coef_, index=feature_cols, columns=["coef"])
non_zero_features = coef_df[coef_df["coef"] != 0]
non_zero_features.sort_values("coef", ascending=False)

,coef
manegement,61.260856
東京,32.373242
裁量労働,24.885839
Vue.js,20.760981
Go言語,19.490601
副業OK,18.783706
ノートPC＋モニタ別途支給,17.859801
イヤホンOK,9.094499
1920x1200以上のモニター環境を提供,7.874984
Python2,7.662115


In [59]:
train

,company_id,position,job_board_info,language_skillset,framework_skillset,working_conditions,manegement,employee_num,location,salary_min,...,時短勤務可,オンライン面談可,服装自由,フリーソフト利用可,原則定時退社,既卒可,イヤホンOK,裁量労働,副業OK,日本語がネイティブレベルでなくても可
0,Cd346dca,サーバサイドエンジニア,"{'employee_num': 26.0, 'location': '東京都千代田区', ...",PHP/Python3/JavaScript,CakePHP/jQuery,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/残業３０H以内/フリードリンク/女性エン...,1,26.0,東京都千代田区,510.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,C713a7cd,プログラマ,"{'employee_num': 17.0, 'location': '東京都立川市', '...",Java/C/VB/JavaScript/HTML+CSS,,残業３０H以内/日本語がネイティブレベルでなくても可,0,17.0,東京都立川市,300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,C1000ca5,サーバーサイドエンジニア,"{'employee_num': 76.0, 'location': '東京都目黒区', '...",Go言語/PHP/Ruby/HTML5+CSS3/HTML+CSS,CodeIgniter/RubyonRails,服装自由/イヤホンOK/ノートPC＋モニタ別途支給/裁量労働/フリードリンク/若手歓迎/オン...,0,76.0,東京都目黒区,400.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,C10010c0,バックエンドエンジニア,"{'employee_num': 53.0, 'location': '大阪府大阪市', '...",PHP/JavaScript/HTML5+CSS3/HTML+CSS/SQL,FuelPHP/jQuery/Vue.js,服装自由/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別途支給/192...,0,53.0,大阪府大阪市,350.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,C94ef74d,CTO候補,"{'employee_num': 30.0, 'location': '東京都新宿区', '...",PHP/Ruby/JavaScript/HTML5+CSS3/SQL,CodeIgniter/Smarty/RubyonRails/jQuery,服装自由/イヤホンOK/フリーソフト利用可/ノートPC＋モニタ別途支給/1920x1200以...,1,30.0,東京都新宿区,600.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,C1891d63,ネイティブエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/Objective-C,AndroidSDK/iOSSDK,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1199,C1892d63,iOSアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Objective-C/Swift,iOSSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1200,C1893d63,サーバーサイドエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java/C/C++/C＃/PHP/Ruby/Perl,Unity,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,1,15.0,東京都港区,500.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1201,C1894d63,Androidアプリエンジニア,"{'employee_num': 15.0, 'location': '東京都港区', 's...",Java,AndroidSDK/Unity/cocos2d,服装自由/イヤホンOK/社長が現役or元エンジニア/フリーソフト利用可/ノートPC＋モニタ別...,0,15.0,東京都港区,300.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [60]:
# 先ほどのlasso回帰の重要度をDataFrameに変換し、0ではなかった要素を抽出します。
coef_df = pd.DataFrame(
    simple_lasso.coef_, index=feature_cols, columns=["coef"])
non_zero_features = coef_df[coef_df["coef"] != 0]
non_zero_features

,coef
manegement,61.260856
employee_num,-0.003050
Vue.js,20.760981
.NETFramework,-1.415389
Unity,-13.106665
JavaScript,-6.508567
Java_x,-12.711513
HTML5+CSS3,-4.824459
Python3,1.808642
Python2,7.662115


In [61]:
# 特徴量をリスト化し、useful_colsに保存します。
useful_cols = list(non_zero_features.reset_index()["index"])
useful_cols

['manegement',
 'employee_num',
 'Vue.js',
 '.NETFramework',
 'Unity',
 'JavaScript',
 'Java_x',
 'HTML5+CSS3',
 'Python3',
 'Python2',
 'C',
 'Node.js_y',
 'Go言語',
 '東京',
 'フロントエンドエンジニア',
 '若手歓迎',
 'ノートPC＋モニタ別途支給',
 '経験が浅い方歓迎',
 '一部在宅勤務可',
 '第二新卒歓迎',
 '社長が現役or元エンジニア',
 '1920x1200以上のモニター環境を提供',
 'オンライン面談可',
 'フリーソフト利用可',
 'イヤホンOK',
 '裁量労働',
 '副業OK']

In [62]:
# useful_colsの要素をカラムにもつデータでの予測を行います。
X_s = train[useful_cols]
y_s = train[target_col]

X_train_s, X_valid_s, y_train_s, y_valid_s = train_test_split(
    X_s, y_s, test_size=0.30, random_state=1234)

# testから予測に使うカラムのみ抽出します。
X_test_s = test[useful_cols]

In [63]:
rf_params = {'n_estimators': 200, 'max_depth': 10, 'random_state': 1234}
f_eng_rf = RandomForestRegressor(**rf_params)
f_eng_rf.fit(X_train_s, y_train_s)
y_pred = f_eng_rf.predict(X_valid_s)
RMSE = np.sqrt(mean_squared_error(y_valid_s , y_pred))
print(RMSE)
RMSE_dic["selected_f_rf"] = RMSE

80.41089314299803


In [64]:
selected_f_lasso = Lasso(random_state=1234)
selected_f_lasso.fit(X_train_s, y_train_s)
y_pred = selected_f_lasso.predict(X_valid_s)
RMSE = np.sqrt(mean_squared_error(y_valid_s, y_pred))
print(RMSE)
RMSE_dic["selected_f_lasso"] = RMSE

78.49830101769935


In [65]:
# 最も精度が高かったモデルの確認
for key, value in RMSE_dic.items():
    if value == min(RMSE_dic.values()):
        print("RMSEが最小となるモデルは", key, "です。")

RMSEが最小となるモデルは simple_rf です。


In [66]:
RMSE_dic

{'simple_rf': 77.83417716266158,
 'simple_lasso': 78.49829937541972,
 'selected_f_rf': 80.41089314299803,
 'selected_f_lasso': 78.49830101769935}

In [67]:
# ランダムサーチにより精度が高くなるn_estimatorsとmax_depthの値を大雑把に探します。
# その後より最適な値をグリッドサーチにより見つけていきます。
# ランダムサーチとグリッドサーチに必要なライブラリをインポートします。
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint

In [68]:
rf = RandomForestRegressor(random_state=1234)
paramR = {'n_estimators': randint(150, 300),
          'max_depth': randint(10, 20)}

# RandomizedSearchCVを初期化します。
# 使用するモデルをestimator、評価方法をscoring、交差検定の分割数をcv、ランダムサーチ特有のパラメータをparam_distributionsで指定します。
RFC_rand = RandomizedSearchCV(
    estimator=rf, param_distributions=paramR, scoring='neg_mean_squared_error', cv=3, random_state=1234)

RFC_rand.fit(X_train_s, y_train_s)

# RFC_rand.best_estimator_で最適化したパラメータを確認できます。
print('ランダムサーチ・ランダムフォレストモデルにおける n_estimators  :  %d' %
      RFC_rand.best_estimator_.n_estimators)

print('ランダムサーチ・ランダムフォレストモデルにおける max_depth  :  %d' %
      RFC_rand.best_estimator_.max_depth)

ランダムサーチ・ランダムフォレストモデルにおける n_estimators  :  226
ランダムサーチ・ランダムフォレストモデルにおける max_depth  :  10


In [69]:
# ランダムサーチで探索したハイパーパラメータ の周辺で再度探索します。
# グリッドサーチに用いるパラメータを辞書型のデータで設定します。
rf = RandomForestRegressor(random_state=1234)
paramG = {'n_estimators': [209, 219, 229],
          'max_depth': [8, 10, 12]}

# 使用するモデルをestimator、評価方法をscoring、交差検定の分割数をcv、ランダムサーチ特有のパラメータをparam_gridで指定します。
RFC_grid = GridSearchCV(
    estimator=rf, param_grid=paramG, scoring='neg_mean_squared_error', cv=3)

RFC_grid.fit(X_train_s, y_train_s)

# RFC_grid.best_estimator_で最適化したパラメータを確認できます。
print('グリッドサーチ・ランダムフォレストモデルにおける n_estimators   :  %d' %
      RFC_grid.best_estimator_.n_estimators)

print('グリッドサーチ・ランダムフォレストモデルにおける max_depth   :  %d' %
      RFC_grid.best_estimator_.max_depth)

グリッドサーチ・ランダムフォレストモデルにおける n_estimators   :  229
グリッドサーチ・ランダムフォレストモデルにおける max_depth   :  8


In [70]:
# グリッドサーチを用いて最適化したパラメータでランダムフォレストを学習します。
rf_params = {'n_estimators': 229, 'max_depth': 8, 'random_state': 1234}
hypra_rf = RandomForestRegressor(**rf_params)
hypra_rf.fit(X_train_s, y_train_s)
y_pred = hypra_rf.predict(X_valid_s)
RMSE = np.sqrt(mean_squared_error(y_valid_s, y_pred))
print(RMSE)
RMSE_dic["hypra_rf"] = RMSE

80.41253309852576


In [71]:
# 最も精度が高かったモデルの確認
for key, value in RMSE_dic.items():
    if value == min(RMSE_dic.values()):
        print("RMSEが最小となるモデルは", key, "です。")

RMSEが最小となるモデルは simple_rf です。


In [72]:
# hypra_rfのRMSEが、最も小さいのでhypra_rfを選択
y_pred = hypra_rf.predict(X_test_s)
output = pd.DataFrame()
output["company_id"] = test["company_id"]
output["salary_min"] = y_pred
output.to_csv("sample.csv", index=False)


In [ ]:
pip install h2o

     |████████████████████████████████| 164.8MB 74kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.1-py2.py3-none-any.whl size=164840229 sha256=4072bf2449991f5a7d497b4f899c7a27ec8ddc95bf6e9a76ef11d678dd275e31
  Stored in directory: /root/.cache/pip/wheels/4d/94/84/02ea21db1eebcef2da872861ce0db478788e27d24bcd5394fb
Successfully built h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpaqega332
  JVM stdout: /tmp/tmpaqega332/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpaqega332/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,"7 days, 11 hours and 53 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_dscy6w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
train = pd.merge(X_train_f, y_train_f, left_index=True, right_index=True)
valid = pd.merge(X_valid_f, y_valid_f, left_index=True, right_index=True)

h2o_train = h2o.H2OFrame(train)
h2o_valid = h2o.H2OFrame(valid)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
# max_runtime_secsは最大実行時間を決めることができます。
aml = H2OAutoML(max_runtime_secs=30)
# 説明変数のカラムを指定します。
x = feature_cols
# 目的変数のカラムを指定します。
y = "salary_min"

# モデルを作成する際に使用したデータを指定します。
# leaderboard_frameでスコアを計算する際に使用したいデータを指定します。
aml.train(x=x, y=y,
          training_frame=h2o_train,
          leaderboard_frame=h2o_valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
#スコアの高いモデル順に並べていきます。
lb = aml.leaderboard
lb.head(rows=lb.nrows)  

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20210402_070213,6056.13,77.8211,6056.13,58.1348,0.178452
StackedEnsemble_AllModels_AutoML_20210402_070213,6076.89,77.9544,6076.89,59.006,0.179524
XGBoost_grid__1_AutoML_20210402_070213_model_1,6341.82,79.6356,6341.82,59.5218,0.18181
GBM_grid__1_AutoML_20210402_070213_model_1,7151.35,84.5656,7151.35,63.8574,0.195853
GBM_3_AutoML_20210402_070213,7457.14,86.3547,7457.14,64.7508,0.19985
GBM_4_AutoML_20210402_070213,7464.46,86.3971,7464.46,64.3894,0.199727
GBM_2_AutoML_20210402_070213,7538.16,86.8226,7538.16,64.8204,0.200851
GBM_5_AutoML_20210402_070213,7540.9,86.8384,7540.9,65.2353,0.200559
DeepLearning_1_AutoML_20210402_070213,7596.54,87.1581,7596.54,61.7935,0.198459
GBM_1_AutoML_20210402_070213,7635.37,87.3806,7635.37,65.8961,0.202124


In [ ]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20210402_070213

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 2941.946220655903
RMSE: 54.23971073536347
MAE: 39.395102677959926
RMSLE: 0.12105202562496403
R^2: 0.6407174694606727
Mean Residual Deviance: 2941.946220655903
Null degrees of freedom: 841
Residual degrees of freedom: 838
Null deviance: 6894626.11520189
Residual deviance: 2477118.7177922702
AIC: 9124.400504912905

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 6486.525367215992
RMSE: 80.53896800441382
MAE: 59.788942264105636
RMSLE: 0.18220458177102916
R^2: 0.20783893601517955
Mean Residual Deviance: 6486.525367215992
Null degrees of freedom: 841
Residual degrees of freedom: 838
Null deviance: 6925809.528308731
Residual deviance: 5461654.359195866
AIC: 9790.132572854718


In [ ]:
# もっとも精度の高かったモデルを用いて予測を行います。
h2o_f_test = h2o.H2OFrame(test[useful_cols])
y_pred = aml.predict(h2o_f_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [ ]:
# "company_id"と予測値である"salary_min"をcsvファイルに保存します。
output = pd.DataFrame()
output["company_id"] = test["company_id"]
output["salary_min"] = h2o.H2OFrame.as_data_frame(y_pred)
output.to_csv("sample_2.csv", index=False)